In [ ]:
import os
import workflow_utils_v3
import sys

from workflow_utils_v3.FileDirectory import Directory

dirs = Directory(rootpath = '/home/mgolub4/DLproj/MLTO_2024/')

# Sets directory of entire package
# rootpath = '/data/tigusa1/MLTO_UCAH/MLTO_2023/'

nbpath = os.path.join(dirs._3_Dynamic_PINN_RNN, 'PINN_training')
cp_dir = os.path.join(nbpath, 'model_CPs')


In [ ]:

# from torchsummary import summary

import pandas as pd
import numpy as np
import json
import glob
import os

# For plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from itertools import cycle
from plotly.colors import sequential, qualitative


import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F


import torch.utils.data as data
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch.optim as optim
# import torchsummary

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from sklearn.model_selection import train_test_split as TTS

In [ ]:
date = '29APR24'
fname_base = f'Dyn_PINN_v0_{date}'

In [ ]:
# dyndb_path = '/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/dyn_stat_database_PINN_ready.csv'
dyndb_path = '/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/dyn_stat_database_PINN_ready_coeffs_scaled.csv'
dyndb = pd.read_csv(dyndb_path)

In [ ]:
# dyndb.columns.values

In [ ]:
# max_min_params = ['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt', 'plateau_stress_g', 'energy_absorbed_g']
# dict_entries = ['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt', 'sig_pl', 'W']


# Modifying from above after calculating sig_pl and W from scaled time series
max_min_params = ['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt'] # 'plateau_stress_g', 'energy_absorbed_g' <<-- in the revised version, these were calculated from the scaled stress/strain series, so pulling their column-wise max-min won't work
dict_entries = ['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt'] # 'sig_pl', 'W'
 
max_min_dict = {}

for par, name in zip(max_min_params, dict_entries):
    data = dyndb[par]
    max = data.max()
    min = data.min()
    max_min_dict[name] = (max, min) 

In [ ]:
idxTr, idxRem = TTS(dyndb, stratify = dyndb['topology_family'], random_state=42, train_size = 0.8)
idxVal, idxTe = TTS(idxRem, random_state = 42, test_size=0.5)

In [ ]:
params = ['volFrac', 
        'CH_11 scaled', 'CH_22 scaled', 'CH_33 scaled', 'CH_44 scaled', 'CH_55 scaled', 'CH_66 scaled',
        'CH_12 scaled', 'CH_13 scaled','CH_23 scaled',
        'EH_11 scaled', 'EH_22 scaled', 'EH_33 scaled',
        'GH_23 scaled', 'GH_13 scaled', 'GH_12 scaled', 
        'vH_12 scaled', 'vH_13 scaled', 'vH_23 scaled', 'vH_21 scaled', 'vH_31 scaled','vH_32 scaled',
        'KH_11 scaled', 'KH_22 scaled', 'KH_33 scaled', 
        'kappaH_11 scaled', 'kappaH_22 scaled', 'kappaH_33 scaled']

In [ ]:
class Stress_Series:
    def __init__(self, series):
        self.series = series
        self.max = series.max()
        self.min = series.min()

    def scale(self):
        return (self.series - self.min) / (self.max - self.min)

In [ ]:
import torch

def min_max_scale_series(batched_time_series):
    # Calculate the maximum and minimum values for each series
    max_values = torch.max(batched_time_series, dim=1)[0]
    min_values = torch.min(batched_time_series, dim=1)[0]

    # Calculate the range for each series
    series_range = max_values - min_values

    # Ensure non-zero range to avoid division by zero
    # series_range = torch.where(series_range == 0, torch.tensor(1e-7), series_range)

    # Min-max scale each series
    scaled_time_series = (batched_time_series - min_values.unsqueeze(1)) / series_range.unsqueeze(1)

    return scaled_time_series


In [ ]:
class PINN_Dataset(Dataset):
    def __init__(self, params, split_dataframe,
                 feat_vec_directory='/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/voxel_embedding_feature_maps', 
                 stress_series_directory='/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/stress_series_data', 
                #  stress_ser_suffix = '_proct_gaus_btrlp_fftlp',
                stress_ser_suffix = '_proct_w_constit_eqn_and_scaled_series',
                #  scale_coeffs_by=[1e9, 1e11, 1, 1, 1],
                 predicted_parameters=True,
                  ):
        self.df = split_dataframe
        self.featvec_dir = feat_vec_directory # for pulling the feature vectors
        self.stress_ser_dir = stress_series_directory # for pulling the time series files
        self.params = params
        self.predicted_parameters = predicted_parameters
        # self.const_eqn_params = ['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt',]
        self.const_eqn_params = ['A_opt scaled', 'B_opt scaled', 'C_opt scaled', 'm_opt scaled', 'n_opt scaled',]

        self.stress_ser_suffix = stress_ser_suffix
        # self.scale_coeffs_by = scale_coeffs_by

        # self.scaler = MinMaxScaler()


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        dyn_series_fname = self.df['dyn_file_name_original'].iloc[idx]

        sig_pl = self.df[self.df['dyn_file_name_original'] == dyn_series_fname]['plateau_stress_g scaled'].values[0].astype(np.float32)
        W = self.df[self.df['dyn_file_name_original'] == dyn_series_fname]['energy_absorbed_g scaled'].values[0].astype(np.float32)

        # feature vector from convolutional neural network convolutional layers output
        featvec_fname = self.df['conv_feat_vec'].iloc[idx] + '.npy'
        featvec_path = os.path.join(self.featvec_dir, featvec_fname)
        featvec = np.load(featvec_path)
        featvec = np.squeeze(featvec, axis=0).astype(np.float32)

        # constitutive equation parameters
        constit_eqn_coeffs = np.asarray(self.df[self.const_eqn_params].iloc[idx])

        # for i, scaler in enumerate(self.scale_coeffs_by):
        #     constit_eqn_coeffs[i] = constit_eqn_coeffs[i] / scaler
            


        # predicted parameters
        if self.predicted_parameters:
            paramvec = np.asarray([self.df[f'pred {par}'].iloc[idx] for par in self.params]).astype(np.float32)
        else:
            paramvec = np.asarray([self.df[f'{par}'].iloc[idx] for par in self.params]).astype(np.float32)

        # stress_series -- for now (April 24), Imma use the truncated datasets, because I think padded batches for RNNs in pytorch will take care of differing lengths
        stress_ser_fname = dyn_series_fname + self.stress_ser_suffix
        stress_ser_path = os.path.join(self.stress_ser_dir, stress_ser_fname+'.csv')
        # stress_series = np.asarray(pd.read_csv(stress_ser_path)['stress_bottom_gsreg']).astype(np.float32)
        stress_series = np.asarray(pd.read_csv(stress_ser_path)['stress_bottom_gsreg_scaled']).astype(np.float32)
        
        stress_series = Stress_Series(stress_series)
        # scaled_stress_series = stress_series.scale()
        stress_series_dic = {'stress_series': stress_series.series, 'max': stress_series.max, 'min': stress_series.min}

        # stress_series_dic = {'stress_series': scaled_stress_series, 'max': stress_series.max, 'min': stress_series.min}

        strain = np.asarray(pd.read_csv(stress_ser_path)['Strain']).astype(np.float32)
                   

        return featvec, paramvec, constit_eqn_coeffs, W, sig_pl, strain, stress_series_dic




In [ ]:
# return featvec, paramvec, stress_series, constit_eqn_coeffs, W, sig_pl, strain
# return nonseries_list, padded_strain_ser, padded_stress_ser

from torch.nn.utils.rnn import pad_sequence

def padded_collate(batch):
    # get list of stress series -- uneven lengths, but will be padded at end
    stress_ser = [torch.tensor(item[-1]['stress_series'], requires_grad=True) for item in batch]
    # get max and min values as a list of lists, where the first value is the maximum and second is the minimum value of the accompanying stress series
    max_mins = [[item[-1]['max'],item[-1]['min']] for item in batch]
    max_mins = np.asarray(max_mins)

    # get strain series, convert to tensor
    strain_ser = [torch.tensor(item[-2], requires_grad=True) for item in batch]
    
    # get other data (featvec, paramvec, constit_eqn_coeffs, W, sig_pl) from data
    nonserbatch = [item[0:-2] for item in batch] # all until last element (stress series) of list given by PINN_Dataset

    nonseries_list = []

    # nonser_len = len(nonserbatch)
    for i in range(5):
        data_list = []
        for data in nonserbatch:
            # print(type(data), len(data))
            data_list.append(data[i])
        data_list = torch.tensor(np.asarray(data_list), requires_grad=True)
        nonseries_list.append(data_list)

    
    padded_stress_ser = pad_sequence(stress_ser, batch_first=True, padding_value=0)
    padded_strain_ser = pad_sequence(strain_ser, batch_first=True, padding_value=0)


    return nonseries_list, padded_strain_ser, padded_stress_ser, max_mins

In [ ]:
batch_size = 4

trdata = PINN_Dataset(params, idxTr)
trloader = DataLoader(trdata, batch_size=batch_size, collate_fn = padded_collate, shuffle=True)

valdata = PINN_Dataset(params, idxVal)
valloader = DataLoader(valdata, batch_size=batch_size, collate_fn = padded_collate, shuffle=True)

tedata = PINN_Dataset(params, idxTe)
teloader = DataLoader(tedata, batch_size=batch_size, collate_fn = padded_collate, shuffle=False)

In [ ]:
# pd.Series(trdata.__getitem__(0)[0][:]).describe()

In [ ]:
# dataset    returns:   featvec, paramvec, constit_eqn_coeffs, W, sig_pl, strain,               stress_series_dic

# dataloader returns:   nonseries_list,                                   padded_strain_ser,    padded_stress_ser, max_mins

In [ ]:
# dyndb.columns.values

In [ ]:
class PINN_loss(nn.Module):

    def __init__(self, coeff_max_min_dict = max_min_dict, offset = -0.01):
        super(PINN_loss, self).__init__()

        self.offset = offset
        self.coeff_max_min_dict = coeff_max_min_dict



    def forward(self, stress_series_predicted, constitutive_equation_coefficients_predicted, constitutive_equation_coefficients_data, 
                 W_data, sig_pl_data, strain): # , , stress_max_min_array, <<-- don't think I need these because I'm calculating the loss as scaled
        
        stress_series_eqn_calculated_scaled_output = self.constitutive_equation(constitutive_equation_coefficients_predicted, strain)
        # print(stress_series_eqn_calculated_scaled[0].shape)
        stress_series_eqn_calculated_scaled = stress_series_eqn_calculated_scaled_output[0].T

        predicted_sig_pl = self.eqn_predict_sig_pl(stress_series_eqn_calculated_scaled)

        W_pred = self.eqn_predicted_W(stress_series_predicted, strain)
        print(f"W_pred: {W_pred.detach().cpu().numpy()}\tSig_pl: {predicted_sig_pl.detach().cpu().numpy()}")
        
        W_loss = nn.L1Loss()(W_pred, W_data)

        sig_pl_loss = nn.L1Loss()(predicted_sig_pl, sig_pl_data)
        

        constit_equation_coeff_loss = nn.L1Loss()(constitutive_equation_coefficients_predicted, constitutive_equation_coefficients_data) #, self.offset, self.coeff_max_min_dict

        stress_series_predicted = torch.squeeze(stress_series_predicted, dim=-1)
        physics_loss = nn.L1Loss()(stress_series_predicted, stress_series_eqn_calculated_scaled)

        total_loss = W_loss + sig_pl_loss + constit_equation_coeff_loss + physics_loss

        output_dic = {
            'total_loss': total_loss,
            'W_loss': W_loss,
            'sig_pl loss': sig_pl_loss,
            'constit_equation_coeff_loss': constit_equation_coeff_loss,
            'physics_loss': physics_loss,
            'stress_series_eqn_calculated_scaled':stress_series_eqn_calculated_scaled, 
            'predicted_sig_pl': predicted_sig_pl, 
            'W_pred': W_pred, 
            'stress_series_predicted': stress_series_predicted, 
            'stress_series_eqn_calculated_scaled_output':stress_series_eqn_calculated_scaled_output
        }

        return total_loss, W_loss, sig_pl_loss, constit_equation_coeff_loss, physics_loss, stress_series_eqn_calculated_scaled, predicted_sig_pl, W_pred, stress_series_predicted, stress_series_eqn_calculated_scaled_output, output_dic
# return total_loss, W_loss, sig_pl_loss, constit_equation_coeff_loss, physics_loss, 
# stress_series_eqn_calculated_scaled, predicted_sig_pl, W_pred, stress_series_predicted, stress_series_eqn_calculated_scaled_output


    def unscale_coeffs(self, coeffs, coeff_max_min_dict, coeff_list =['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt',]):
        unscaled_coeffs = torch.zeros_like(coeffs)

        for i in range(coeffs.shape[0]):
            for j, name in enumerate(coeff_list):
                max = coeff_max_min_dict[name][0]
                min = coeff_max_min_dict[name][1]

                unscaled_coeffs[i][j] = (coeffs[i][j] * (max - min)) + min

        return unscaled_coeffs
    
    def unscale_dyn_params(array, coeff_max_min_dict, dyn_param = 'sig_pl'):
        max = coeff_max_min_dict[dyn_param][0]
        min = coeff_max_min_dict[dyn_param][1]

        array = array*(max - min) + min

        return array
    
    def constitutive_equation(self, coeffs_pred, strain):

        coeffs_pred_unscaled = self.unscale_coeffs(coeffs_pred, self.coeff_max_min_dict)
        A_pred = coeffs_pred_unscaled[:, 0]
        B_pred = coeffs_pred_unscaled[:, 1]
        C_pred = coeffs_pred_unscaled[:, 2]
        m_pred = coeffs_pred_unscaled[:, 3]
        n_pred = coeffs_pred_unscaled[:, 4]

        mask = torch.ge(strain.T, self.offset*-1)
        strain = strain.T + self.offset
        strain = torch.where(mask, strain, torch.tensor(0))


        # stress_series_eqn_calculated = A_pred * (strain.T + self.offset)**m_pred + B_pred*((strain.T + self.offset)/(C_pred-(strain.T + self.offset)))**n_pred
        stress_series_eqn_calculated = A_pred * (strain + self.offset)**m_pred + B_pred*((strain + self.offset)/(C_pred-(strain + self.offset)))**n_pred

        stress_series_eqn_calculated = torch.nan_to_num(stress_series_eqn_calculated, nan = 0.0)


        max = torch.max(stress_series_eqn_calculated, dim=0)[0]

        min = torch.min(stress_series_eqn_calculated, dim=0)[0]

        stress_ser_range = max - min

        # stress_ser_eqn_calculated_scaled = (stress_series_eqn_calculated - min.unsqueeze(1)) / stress_ser_range.unsqueeze(1)
        stress_ser_eqn_calculated_scaled = (stress_series_eqn_calculated - min.unsqueeze(0)) / stress_ser_range.unsqueeze(0)

        out_dic = {
            'stress_ser_eqn_calculated_scaled': stress_ser_eqn_calculated_scaled,
            'coeffs_pred_unscaled': coeffs_pred_unscaled,
            'coeffs_pred': coeffs_pred,
            'max': max,
            'min': min,
            'stress_ser_range': stress_ser_range,
            'stress_series_eqn_calculated': stress_series_eqn_calculated,
            'strain': strain
        }


        return stress_ser_eqn_calculated_scaled, coeffs_pred_unscaled, coeffs_pred, max, min, stress_ser_range, stress_series_eqn_calculated, strain, out_dic
        
        # return stress_series_eqn_calculated
    
    def eqn_predict_sig_pl(self, stress_series_eqn_calculated): # CALLED
         
        sig_pl_pred = torch.mean(stress_series_eqn_calculated[:, 200:400], dim=1)

        return sig_pl_pred
    
    def eqn_predicted_W(self, stress_pred, strain): # CALLED
        stress_start_idx = int(-1*self.offset*1e3)
        stress_pred = torch.squeeze(stress_pred, dim=-1)
        # print(strain.shape, stress_pred.shape)
        W_pred = torch.trapz(stress_pred[:, stress_start_idx:], strain[:, stress_start_idx:], dim=1)
        # print(f"W_pred shape: {W_pred.shape}")
        return W_pred

In [ ]:
class Dynamic_Stress_PINN(nn.Module):
    
    def __init__(self, params, hidden_size=256, num_lstm_layers=4, lstm_output_dim=1):
        numparams = len(params)
        input_vec_dim = 1024 + numparams
        linear_out_dims = 5
        # self.series_in_dim = series_input_dim
        self.hidden_size = hidden_size
        self.num_lstm_layers = num_lstm_layers
        self.lstm_output_dim = lstm_output_dim
        super(Dynamic_Stress_PINN, self).__init__()

        self.stress_ser_predictor = nn.LSTM(input_vec_dim, hidden_size, num_lstm_layers, batch_first=True)
        self.lstm_linear = nn.Sequential(nn.Linear(hidden_size, lstm_output_dim), nn.ReLU())


        self.constit_eqn_coeff_predictor = nn.Sequential(
            nn.Linear(input_vec_dim, 1024),
            nn.Linear(1024, 512),nn.ReLU(),
            nn.Linear(512, 256),nn.ReLU(),
            nn.Linear(256, 128),nn.ReLU(),
            nn.Linear(128, linear_out_dims)
        )


    def forward(self, feature_vec, property_vec, strain_series):
        input_vec = torch.cat([feature_vec, property_vec], dim=1)

        batch_size = strain_series.size(0)
        h = torch.randn(self.num_lstm_layers, self.hidden_size).to(strain_series.device)
        c = torch.randn(self.num_lstm_layers, self.hidden_size).to(strain_series.device)
        
        stress_ser = []

        for i in range(strain_series.size(1)):
            stress, (h, c) = self.stress_ser_predictor(input_vec, (h, c))
            stress = self.lstm_linear(stress)
            stress_ser.append(stress)
       
        stress_ser = torch.stack(stress_ser, dim=1)

        constit_eqn_coeffs = self.constit_eqn_coeff_predictor(input_vec)
        print(f'constit_eqn_coeffs predicted\n{constit_eqn_coeffs.detach().cpu().numpy()}')

        return stress_ser, constit_eqn_coeffs


In [ ]:
class Dynamic_Stress_PINN2(nn.Module):
    
    def __init__(self, params, hidden_size=256, num_lstm_layers=4, lstm_output_dim=1):
        numparams = len(params)
        input_vec_dim = 1024 + numparams
        linear_out_dims = 5
        # self.series_in_dim = series_input_dim
        self.hidden_size = hidden_size
        self.num_lstm_layers = num_lstm_layers
        self.lstm_output_dim = lstm_output_dim
        super(Dynamic_Stress_PINN2, self).__init__()

        self.stress_ser_predictor = nn.LSTM(input_vec_dim, hidden_size, num_lstm_layers, batch_first=True)
        self.lstm_linear = nn.Sequential(nn.BatchNorm1d(hidden_size), nn.Linear(hidden_size, lstm_output_dim),  nn.ReLU())


        # self.constit_eqn_coeff_predictor = nn.Sequential(
        #     nn.Linear(input_vec_dim, 512),
        #     nn.Linear(512, 64),nn.ReLU(),
        #     nn.Linear(64, linear_out_dims)
        # )
        self.constit_eqn_coeff_predictor = nn.Sequential(
            nn.Linear(input_vec_dim, 512),
            nn.BatchNorm1d(512),  # BatchNorm1d for 1D input (e.g., after linear layers)
            nn.ReLU(),
            nn.Linear(512, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, linear_out_dims),
            nn.ReLU()
        )

        for module in self.modules():
            if isinstance(module, nn.LSTM):
                for name, param in module.named_parameters():
                    if 'weight' in name:
                        init.orthogonal_(param)

            elif isinstance(module, nn.Linear):
                init.uniform_(module.weight, a=0, b=1)


    def forward(self, feature_vec, property_vec, strain_series):
        input_vec = torch.cat([feature_vec, property_vec], dim=1)

        batch_size = strain_series.size(0)
        h = torch.randn(self.num_lstm_layers, self.hidden_size).to(strain_series.device)
        c = torch.randn(self.num_lstm_layers, self.hidden_size).to(strain_series.device)
        
        stress_ser = []

        for i in range(strain_series.size(1)):
            stress, (h, c) = self.stress_ser_predictor(input_vec, (h, c))
            stress = self.lstm_linear(stress)
            stress_ser.append(stress)
       
        stress_ser = torch.stack(stress_ser, dim=1)

        constit_eqn_coeffs = self.constit_eqn_coeff_predictor(input_vec)
        print(f'constit_eqn_coeffs predicted\n{constit_eqn_coeffs.detach().cpu().numpy()}')

        return stress_ser, constit_eqn_coeffs

In [ ]:
# pinn = Dynamic_Stress_PINN(params).to(device)

In [ ]:
pinn2 = Dynamic_Stress_PINN2(params).to(device)

In [ ]:
# from torchinfo import summary
# summary(pinn)


In [ ]:
# dataset returns-->         featvec, paramvec, constit_eqn_coeffs, W, sig_pl, || strain,            || stress_series,
# dataloader returns:->     ^--------------nonseries_list------------------^  || padded_strain_ser, || padded_stress_ser

# dataset    returns:   featvec, paramvec, constit_eqn_coeffs, W, sig_pl, strain,               stress_series_dic
# dataloader returns:   nonseries_list,                                   padded_strain_ser,    padded_stress_ser, max_mins


alpha = 1.0
beta = 2.0
gamma = 1.0

def pinn_train(pinn, dataloader, loss_func, optimizer, alpha=alpha, beta=beta, gamma=gamma): #PINN_loss
    pinn.train()  # Set the model to training mode
    # running_loss = 0.0
    total_run_loss = 0.0
    
    dyn_param_run_loss = 0.0
    coeff_run_loss = 0.0
    phys_run_loss = 0.0
    pbar = tqdm(dataloader)  # Use tqdm for progress bars
    for non_series_data, strain_series, stress_series, max_mins in pbar:
        global feature_vec, param_vec, strs_ser_glob, coeffs_glob, strn_ser_glob
        feature_vec = non_series_data[0].cuda()
        param_vec   = non_series_data[1].cuda()
        strs_ser_glob = stress_series = stress_series.cuda()  # Move inputs to GPU
        coeffs_glob = const_eqn_coeffs = non_series_data[2].cuda()

        W = non_series_data[3].cuda()
        # print(f"W shape: {W.shape}")
        sig_pl = non_series_data[4].cuda()
        strain_series = strn_ser_glob = strain_series.cuda()

    
        optimizer.zero_grad()
        global stress_series_pred
        global constit_eqn_coeffs_pred
        stress_series_pred, constit_eqn_coeffs_pred = pinn(feature_vec, param_vec, strain_series)
        # print(stress_series_pred.shape, constit_eqn_coeffs_pred.shape)

        # loss_val, W_loss, sig_pl_loss, const_eqn_coeff_loss, phys_loss, *args = loss_func()(stress_series_pred, constit_eqn_coeffs_pred, const_eqn_coeffs, W, sig_pl, strain_series)
                                                                    # PINN_loss(stress_series_pred, constit_eqn_coeffs_pred, const_eqn_coeffs_data, W_data, sig_pl_data, strain, offset=-0.01):
        #total_loss, W_loss, sig_pl_loss, constit_equation_coeff_loss, physics_loss, stress_series_eqn_calculated_scaled, predicted_sig_pl, W_pred, stress_series_predicted, stress_series_eqn_calculated_scaled_output
        
        global loss_val, W_loss, sig_pl_loss, const_eqn_coeff_loss, phys_loss, stress_series_eqn_calculated_scaled, predicted_sig_pl, W_pred, stress_series_predicted, stress_series_eqn_calculated_scaled_output
        loss_val, W_loss, sig_pl_loss, const_eqn_coeff_loss, phys_loss, stress_series_eqn_calculated_scaled, predicted_sig_pl, W_pred, stress_series_predicted, stress_series_eqn_calculated_scaled_output = loss_func()(stress_series_pred, constit_eqn_coeffs_pred, const_eqn_coeffs, W, sig_pl, strain_series)
        loss_val = alpha*W_loss + alpha*sig_pl_loss + beta*const_eqn_coeff_loss + gamma*phys_loss
        dyn_param_loss = W_loss + sig_pl_loss
        loss_val.backward() #retain_graph=True
        # torch.nn.utils.clip_grad_norm_(pinn.parameters(), max_grad_norm)

        for name, param in pinn.named_parameters():
            if param.grad is not None:
                print(f'Gradient {name}: {param.grad.norm().item()}')
                # print(f'Gradient {name}: {param.grad}')

                
        optimizer.step()
        # running_loss += loss_val.item()
        total_run_loss += loss_val.item()
        dyn_param_run_loss += dyn_param_loss.item()
        coeff_run_loss += const_eqn_coeff_loss.item()
        phys_run_loss += phys_loss.item()
        # pbar.set_description(f'Train Loss: {running_loss / (pbar.n + 1):.4f}')
        pbar.set_description(f'Losses:\tTotal: {total_run_loss / (pbar.n + 1):.4f}\tDyn Params: {dyn_param_run_loss / (pbar.n + 1):.4f}\tCoeff: {coeff_run_loss / (pbar.n + 1):.4f}\tPhysics: {phys_run_loss / (pbar.n + 1):.4f}\t')
        # print("Data index is: f{idx}")
    return total_run_loss / len(dataloader),  dyn_param_loss, const_eqn_coeff_loss, phys_loss

In [ ]:
def pinn_validate(pinn, dataloader, loss_func=PINN_loss):
    pinn.train()  # Set the model to training mode
    running_loss = 0.0
    pbar = tqdm(dataloader)  # Use tqdm for progress bars
    with torch.no_grad():
        for non_series_data, strain_series, stress_series, max_mins in pbar:
            feature_vec = non_series_data[0].cuda()
            param_vec   = non_series_data[1].cuda()
            stress_series = stress_series.cuda()  # Move inputs to GPU
            const_eqn_coeffs = non_series_data[2].cuda()

            W = non_series_data[3].cuda()
            sig_pl = non_series_data[4].cuda()
            strain_series = strain_series.cuda()

            stress_series_pred, constit_eqn_coeffs_pred = pinn(feature_vec, param_vec, strain_series)

            loss_val, dyn_param_loss, const_eqn_coeff_loss, phys_loss = loss_func()(stress_series_pred, constit_eqn_coeffs_pred, const_eqn_coeffs, W, sig_pl, strain_series)
            # return total_loss, loss_data_1, loss_data_2, loss_physics
            
            running_loss += loss_val.item()
            pbar.set_description(f'Train Loss: {running_loss / (pbar.n + 1):.4f}\t ')
        return running_loss / len(dataloader),  dyn_param_loss, const_eqn_coeff_loss, phys_loss


In [ ]:
# Model and training hyper(?)parameters
model = pinn2
EPOCHS = 125

lossfunc = torch.nn.L1Loss() # this is MAE loss
lossfunc_name = 'MAE'
# optimizer = optim.Adam(model.parameters(), lr=0.001)

cp_dir = os.path.join(nbpath, 'model_CPs')

cp_name = f'CP_{fname_base}.pth'
best_weights_path = os.path.join(cp_dir, cp_name)
print(best_weights_path)

In [ ]:
patience = 75

min_val_loss = float('inf')
best_val_loss = float('inf')
early_stop_counter = 0
# earlystop_min_delta = 0.000075
earlystop_min_delta = 0.00075 # For L1Loss (MAE)

# os.makedirs(best_weights_path, exist_ok=True)
best_epoch = 0

train_losses = []
val_losses = []

epochs_completed=0

lrate = 1e-7
optimizer = optim.SGD(model.parameters(), lr=lrate)#optim.Adam(pinn.parameters(), lr=0.00001)
max_grad_norm = 1.0

In [ ]:

# return running_loss / len(dataloader),  dyn_param_loss, const_eqn_coeff_loss, phys_loss
lossfunc = PINN_loss

try:

    for epoch in range(EPOCHS):
        # Train the model
        train_loss = pinn_train(model, trloader, lossfunc, optimizer)
        train_loss_val = train_loss[0]
        train_dyn_param_loss = train_loss[1]
        train_const_eqn_coeff_loss = train_loss[2]
        train_phys_loss = train_loss[3]

        # Validate the model
        val_loss = pinn_train(model, valloader, lossfunc)
        val_loss_val = val_loss[0]
        val_dyn_param_loss = val_loss[1]
        val_const_eqn_coeff_loss = val_loss[2]
        val_phys_loss = val_loss[3]

        print(f'training:\ttotal loss: {train_loss_val:.5f}, dynamic param loss: {train_dyn_param_loss:.5f}\nconstitutive equation coefficient loss: {train_const_eqn_coeff_loss:.5f}, physics_loss: {train_phys_loss:.5f}')
        print(f'validation:\ttotal_loss:{val_loss_val:.5f}, dynamic param loss: {val_dyn_param_loss:.5f}\nconstitutive equation coefficient loss: {val_const_eqn_coeff_loss:.5f}, physics_loss: {val_phys_loss:.5f}')


        # Save the model's weights if validation loss is improved
        improvement_delta = best_val_loss - val_loss_val

        if val_loss_val < best_val_loss:
            pct_improved = (best_val_loss - val_loss) / best_val_loss * 100
            print(f"Val loss improved from {best_val_loss:.5f} to {val_loss:.5f} ({pct_improved:.2f}% improvement) saving model state...")
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_weights_path)  # Save model weights to file
        else:
            print(f'Val loss did not improve from {best_val_loss:.5f}.')
            # early_stop_counter += 1  # Increment early stopping counter

        if improvement_delta > earlystop_min_delta:
            early_stop_counter = 0
        else:
            early_stop_counter +=1


        # Collect model training history
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        # Check for early stopping
        if early_stop_counter >= patience:
            print(f'Validation loss did not improve for {early_stop_counter} epochs. Early stopping...')
            model.load_state_dict(torch.load(best_weights_path))
            print(f"Model best weights restored - training epoch {best_epoch}")
            break

        print(f'Epoch [{epoch+1}/{EPOCHS}]\tTrain Loss: {train_loss_val:.5f}\tValidation Loss: {val_loss_val:.5f}')

        epochs_completed +=1


    # Load the best weights at end of training epochs
    model.load_state_dict(torch.load(best_weights_path))  # Load best model weights
    print(f'Training epochs completed, best model weights restored - epoch {best_epoch}')
    min_val_loss = best_val_loss

except KeyboardInterrupt:
    hist_dict = {f'train_loss {lossfunc_name}': train_losses, f'val_loss {lossfunc_name}': val_losses}
    model.load_state_dict(torch.load(best_weights_path))


In [ ]:
stress_series_eqn_calculated_scaled_output[-1]

In [ ]:
stress_series_eqn_calculated_scaled

In [ ]:
# stress_series_eqn_calculated_scaled_output[-1].T[0,:]

In [ ]:
import plotly.express as px

In [ ]:
batch = next(iter(trloader))

In [ ]:
batch[0][3]

In [ ]:
batch[0][4]

In [ ]:
np.mean(batch[2][:, 200:400].detach().cpu().numpy(), axis=1)

In [ ]:
np.trapz(batch[1][0,:].detach().cpu().numpy(),batch[2][0,:].detach().cpu().numpy())

In [ ]:
np.trapz(batch[1].detach().cpu().numpy(),batch[2].detach().cpu().numpy())

In [ ]:
np.trapz(stress_series_eqn_calculated_scaled.detach().cpu().numpy(), stress_series_eqn_calculated_scaled_output[-1].T.detach().cpu().numpy())


In [ ]:
# px.scatter(x = stress_series_eqn_calculated_scaled_output[-1].T.detach().cpu().numpy()[0,:], y=stress_series_eqn_calculated_scaled.detach().cpu().numpy()[0,:])

In [ ]:
# torch.trapz(stress_series_eqn_calculated_scaled, stress_series_eqn_calculated_scaled_output[-1].T)

In [ ]:
np.where(np.isnan(stress_series_eqn_calculated_scaled_output[-1].cpu().detach().numpy()))

In [ ]:
stress_series_eqn_calculated_scaled

In [ ]:
np.where(np.isnan(stress_series_eqn_calculated_scaled.cpu().detach().numpy()))

In [ ]:
torch.trapz(stress_series_eqn_calculated_scaled_output[-1].T, stress_series_eqn_calculated_scaled)


In [ ]:
stress_series_eqn_calculated_scaled[0,:]

In [ ]:
# stress_series_eqn_calculated_scaled_output[-1].T[0,:]

In [ ]:
strn_ser_glob[0,:]

In [ ]:
torch.trapz(stress_series_eqn_calculated_scaled[0,:], strn_ser_glob[0,:], )

In [ ]:
strn_ser_glob

In [ ]:
# stress_series_eqn_calculated_scaled[0,:]

In [ ]:
# dataset returns-->         featvec, paramvec, constit_eqn_coeffs, W, sig_pl, || strain,            || stress_series,
# dataloader returns:->     ^--------------nonseries_list------------------^  || padded_strain_ser, || padded_stress_ser

# dataset    returns:   featvec, paramvec, constit_eqn_coeffs, W, sig_pl, strain,               stress_series_dic
# dataloader returns:   nonseries_list,                                   padded_strain_ser,    padded_stress_ser, max_mins

In [ ]:
pinn2.constit_eqn_coeff_predictor[0].weight

In [ ]:
batch = next(iter(trloader))

In [ ]:
batch

In [ ]:
out_test2 = pinn2(batch[0][0].cuda(),batch[0][1].cuda(), batch[1].cuda())

In [ ]:
out_test2

In [ ]:
test_loss2 = PINN_loss()(out_test2[0].cpu(), out_test2[1].cpu(), batch[0][2], batch[0][3], batch[0][4], batch[1])

In [ ]:
test_loss2

In [ ]:
test_loss2[-1][-2][:,0]

In [ ]:
# return total_loss, W_loss, sig_pl_loss, constit_equation_coeff_loss, physics_loss, stress_series_eqn_calculated_scaled, predicted_sig_pl, W_pred, stress_series_predicted, stress_series_eqn_calculated_scaled_output

In [ ]:
test_loss2[-1][0]

In [ ]:
test_loss2[0].backward()

In [ ]:
for name, param in pinn2.named_parameters():
    print(f"Gradient {name}:\t {param.grad.shape}\t{param.grad.norm().item()} \t {param.grad}")

In [ ]:
out_test = pinn2(batch[0][0].cuda(),batch[0][1].cuda(), batch[1].cuda())

In [ ]:
out_test

In [ ]:
test_loss = PINN_loss()(out_test[0].cpu(), out_test[1].cpu(), batch[0][2], batch[0][3], batch[0][4], batch[1])

In [ ]:
loss_val.backward(retain_graph=True)

In [ ]:
# test_loss[0].backward()

In [ ]:
for name, param in pinn2.named_parameters():
    print(f"Gradient {name}:\t {param.grad.shape}\t{param.grad.norm().item()} \t {param.grad}")

In [ ]:
test_loss

In [ ]:
invec = torch.cat([batch[0][0], batch[0][1]], dim=1)

In [ ]:
invec.shape

In [ ]:
invec

In [ ]:
batch[0][2]

In [ ]:
pinn2.constit_eqn_coeff_predictor(invec.cuda())

In [ ]:
out = pinn2(batch[0][0].cuda(), batch[0][1].cuda(), batch[1].cuda())

In [ ]:
out[0].mT

In [ ]:
 featvec, paramvec, constit_eqn_coeffs, W, sig_pl, strain, stress_series_dic


# <u> __What have I learned below?__</u> __what was the question I was pursuing anyway?__
## Answer: 
### Why is W_pred (predicted energy absorbed) __EVER__ less than 0???
### Answer to the answer:
# BECAUSE OF torch (or numpy) .trapz
### BECAUSE, the way I've padded my data, specifically my strain values, when it gets to the end, it goes from the highest strain value in the series, to 0.00
### Trapz calculates based on $(x_{i} - x_{i-1}) /2$ , which means that if x_{i} is 0.000 and x_{i-1} is NOT, i.e., is like 0.590, then the calculated value of $\Delta y$
### is multiplied by a negative number
# __TO FIX THIS__, I am going to, above, find the index for each strain series where the padding starts, and then when calculating .trapz, break out each series and calculate
## .trapz [:padding_start_index]

In [ ]:
lossval = PINN_loss()(out[0], out[1], batch[0][2].cuda(), batch[0][3].cuda(), batch[0][4].cuda(), batch[1].cuda())

In [ ]:
max_min_dict

In [ ]:
out[1]

In [ ]:
lossval[-1]['predicted_sig_pl']

In [ ]:
lossval[-1].keys()

In [ ]:
np.any(np.array([[ 0,  1,  2, -3,  3],
                    [-1,  1,  2, -1,  1]]) <0)

In [ ]:
np.any(lossval[-1]['stress_series_predicted'].detach().cpu().numpy() <0)

In [ ]:
strntest = lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['strain'][:,0]

In [ ]:
padded_series = torch.tensor([[1.0, 2.0, 3.0, 0.0, 0.0],
                               [4.0, 5.0, 0.0, 0.0, 0.0]])
x_series = torch.tensor([[1.0, 2.0, 3.0, 4.0, 5.0],
                         [1.0, 2.0, 3.0, 4.0, 5.0]])

# Identify the start index of the padding
start_of_padding = (padded_series == 0).sum(dim=1)

In [ ]:
start_of_padding

In [ ]:
# (strntest == 0).sum(dim=-1)
np.argmax(strntest[100:].detach().cpu().numpy() == 0)

In [ ]:
strntest[580:590]

In [ ]:
lossval[-1]['W_pred']

In [ ]:
lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['strain'][580:584,0] # torch.Size([618, 4])

In [ ]:
lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['stress_ser_eqn_calculated_scaled'][580:584,0]#.shape#[:,580:583] #torch.Size([618, 4])

In [ ]:
y = lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['stress_ser_eqn_calculated_scaled'][:584,0].detach().cpu().numpy()
x = lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['strain'][:584,0].detach().cpu().numpy()

In [ ]:
y2 = lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['stress_ser_eqn_calculated_scaled'][:583,0].detach().cpu().numpy()
x2 = lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['strain'][:583,0].detach().cpu().numpy()

In [ ]:
# Initialize variables to store running total and previous x, y values
trapz_sum2 = 0.0
prev_x = x2[0]
prev_y = y2[0]

trapz_values2 = []
running_totals2 = []

# Loop through the arrays
for i in range(1, len(x2)):
    # Current x, y values
    current_x = x2[i]
    current_y = y2[i]

    # Calculate the trapezoidal rule for this segment
    trapz_segment2 = (current_x - prev_x) * (prev_y + current_y) / 2
    
    # Update the running total
    trapz_sum2 += trapz_segment2
    
    # Append the incremental calculation and running total to the lists
    trapz_values2.append(trapz_segment2)
    running_totals2.append(trapz_sum2)
    
    # Update previous x, y values for the next iteration
    prev_x = current_x
    prev_y = current_y

In [ ]:
trapz_sum2

In [ ]:
torch.trapz(lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['stress_ser_eqn_calculated_scaled'][:583,0], lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['strain'][:583,0], dim=0)

In [ ]:
lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['strain'][580:585,3]

In [ ]:
lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['strain'][580:585,0]


In [ ]:
lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['stress_ser_eqn_calculated_scaled'][580:585,0]

In [ ]:
torch.trapz(lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['stress_ser_eqn_calculated_scaled'][:585,0], lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['strain'][:585,3], dim=0).item()

In [ ]:
torch.trapz(lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['stress_ser_eqn_calculated_scaled'][:583,0], lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['strain'][:583,3], dim=0).item()

In [ ]:
np.trapz(lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['stress_ser_eqn_calculated_scaled'][:,0].detach().cpu().numpy(), lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['strain'][:,3].detach().cpu().numpy())

In [ ]:
# running_totals2

In [ ]:
# Initialize variables to store running total and previous x, y values
trapz_sum = 0.0
prev_x = x[0]
prev_y = y[0]

trapz_values = []
running_totals = []

# Loop through the arrays
for i in range(1, len(x)):
    # Current x, y values
    current_x = x[i]
    current_y = y[i]

    # Calculate the trapezoidal rule for this segment
    trapz_segment = (current_x - prev_x) * (prev_y + current_y) / 2
    
    # Update the running total
    trapz_sum += trapz_segment
    
    # Append the incremental calculation and running total to the lists
    trapz_values.append(trapz_segment)
    running_totals.append(trapz_sum)
    
    # Update previous x, y values for the next iteration
    prev_x = current_x
    prev_y = current_y

In [ ]:
# trapz_values

In [ ]:
torch.trapz(lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['stress_ser_eqn_calculated_scaled'][:,:583], lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['strain'][:,:583], dim=0)


In [ ]:
torch.trapz(lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['stress_ser_eqn_calculated_scaled'][:,0], lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['strain'][:,3], dim=0)


In [ ]:
torch.trapz(lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['stress_ser_eqn_calculated_scaled'], lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['strain'], dim=0)

In [ ]:
lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1].keys()

In [ ]:
lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['stress_ser_eqn_calculated_scaled']

In [ ]:
np.where(lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['stress_ser_eqn_calculated_scaled'].detach().cpu().numpy() < 0)

In [ ]:
torch.mean(lossval[-1]['stress_series_eqn_calculated_scaled_output'][-1]['stress_ser_eqn_calculated_scaled'][200:400, :], axis=0)

In [ ]:
lossval[5][0,:]

In [ ]:
for i, v in pinn2.named_parameters():
    print(i, v)

In [ ]:
next(iter(pinn2.named_parameters()))[1].grad

In [ ]:
for name, param in pinn2.named_parameters():
    if param is not None:
        print(f"Gradient {name}:\t {param.grad.shape}\t{param.grad.norm().item()} \t {param.grad}")

In [ ]:
# global stress_series_eqn_calculated_scaled, predicted_sig_pl, W_pred, stress_series_predicted, stress_series_eqn_calculated_scaled_output

In [ ]:
stress_series_eqn_calculated_scaled_output[2]

In [ ]:
stress_series_eqn_calculated_scaled_output[-3]

In [ ]:
torch.max(stress_series_eqn_calculated_scaled_output[-1], dim=0)[0] - torch.min(stress_series_eqn_calculated_scaled_output[-1], dim=0)[0]

In [ ]:
torch.min(stress_series_eqn_calculated_scaled_output[-1], dim=0)


In [ ]:
import plotly.express as px

In [ ]:
stress_series_test = stress_series_eqn_calculated_scaled_output[-2][:,0].detach().cpu().numpy()

In [ ]:
strain_series_test = stress_series_eqn_calculated_scaled_output[-1][:,0].detach().cpu().numpy()

In [ ]:
px.scatter(x = strain_series_test, y = stress_series_test)

In [ ]:
stress_series_test = stress_series_eqn_calculated_scaled_output[-2][:,1].detach().cpu().numpy()

In [ ]:
strain_series_test = stress_series_eqn_calculated_scaled_output[-1][:,1].detach().cpu().numpy()

In [ ]:
px.scatter(x = strain_series_test, y = stress_series_test)

In [ ]:
torch.mean(stress_series_eqn_calculated_scaled[:, 200:400], dim=1)

In [ ]:
torch.max(lossfunc().constitutive_equation(coeffs_glob, strn_ser_glob)[-1], dim=0)

In [ ]:
lossfunc().constitutive_equation(coeffs_glob, strn_ser_glob)

In [ ]:
lossfunc().constitutive_equation(coeffs_glob, strn_ser_glob)[0][:,0]

In [ ]:

hist_dict = {f'train_loss {lossfunc_name}': train_losses, f'val_loss {lossfunc_name}': val_losses}

histno=1
histpath = os.path.join(nbpath,'model_jsons',f'{cp_name[:-4]}_training_history{histno}_{epochs_completed}ep.json')

pinn.eval()

with open(histpath, 'w') as f:
    json.dump(hist_dict, f)